In [6]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

# Configuração inicial do WebDriver
service = Service()
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
options.add_argument("--disable-notifications")
options.add_argument('--no-sandbox')
options.add_argument('--verbose')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Inicie o navegador
browser = webdriver.Chrome(service=service, options=options)

# Lista de URLs das ações
acoes_urls = ['https://investidor10.com.br/acoes/taee11/', 'https://investidor10.com.br/acoes/wege3/', 'https://investidor10.com.br/acoes/petr4/']

df_final = pd.DataFrame()

for url in acoes_urls:
    browser.get(url)
    time.sleep(10)
    page_content = browser.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    table = soup.find(id='table-indicators-history')

    if table:
        years = [th.get_text().strip() for th in table.find_all("th", class_="year")]
        indicadores = [td.get_text().strip() for td in table.find_all("td", class_="indicator")]

        all_values = []
        for row in table.find_all("tr", class_=["visible-even", "visible-odd"]):
            values = [value_td.get_text().strip() for value_td in row.find_all("td", class_="value")]
            if len(values) == len(years):
                all_values.append(values)

        df_temp = pd.DataFrame(all_values, columns=years, index=indicadores)
        df_temp['acao'] = url[33:].strip("/")   # Adiciona uma coluna para identificar a ação
        df_final = df_final.append(df_temp)

browser.quit()
print(df_final)

C:\Users\Lucia\AppData\Local\Temp\ipykernel_6708\1234704277.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_temp)
C:\Users\Lucia\AppData\Local\Temp\ipykernel_6708\1234704277.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_temp)
C:\Users\Lucia\AppData\Local\Temp\ipykernel_6708\1234704277.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df_temp)


                       Atual     2022    2021    2020    2019    2018    acao
P/L                    10,15     8,24    5,66    5,07   10,72    7,59  taee11
P/RECEITA (PSR)         5,05     4,56    3,61    3,22    5,98    4,97  taee11
P/VP                    1,77     1,82    1,87    2,16    2,18    1,78  taee11
DIVIDEND YIELD (DY)    9,87%   14,00%  12,38%   9,63%   6,06%  11,81%  taee11
PAYOUT                99,75%  115,40%  70,09%  48,88%  65,01%  89,63%  taee11
...                      ...      ...     ...     ...     ...     ...     ...
PATRIMÔNIO / ATIVOS     0,37     0,37    0,40    0,24    0,32    0,32   petr4
PASSIVOS / ATIVOS       0,62     0,63    0,60    0,76    0,68    0,67   petr4
LIQUIDEZ CORRENTE       0,90     1,00    1,25    1,21    0,97    1,48   petr4
CAGR RECEITAS 5 ANOS  10,70%   17,72%   9,88%  -3,29%  -2,17%   2,79%   petr4
CAGR LUCROS 5 ANOS    42,52%  246,76%      -%      -%      -%   3,02%   petr4

[90 rows x 7 columns]


In [8]:
df_final.to_csv("investidor10_acoes.csv")